In [1]:
import pandas as pd
import json

from bs4 import BeautifulSoup
import re

In [2]:
dir = '../data/raw/Raw_tweets_DonaldTrump_before_elec.json'

df = pd.read_json(open(dir, 'r'))

In [3]:
df.head()

,created_at,text
0,"datetime.datetime(2016, 11, 6, 19, 26, 58)",Thank you Iowa - Get out &amp; #VoteTrumpPence...
1,"datetime.datetime(2016, 11, 6, 18, 9, 44)",RT @IvankaTrump: Thank you New Hampshire! ????...
2,"datetime.datetime(2016, 11, 6, 17, 6, 8)",Van Jones: ‘There Is A Crack in the Blue Wall’...
3,"datetime.datetime(2016, 11, 6, 6, 8, 32)","Great night in Denver, Colorado- thank you! To..."
4,"datetime.datetime(2016, 11, 6, 4, 27)",RT @DanScavino: Join @realDonaldTrump LIVE in ...


In [4]:
# Trying out the lengths of the tweets:
df['pre_clean_len'] = [len(t) for t in df.text]
df['pre_clean_len'][:20]

0      98
1      70
2      97
3     138
4     142
5     135
6      77
7     140
8      39
9      83
10    108
11    112
12    139
13    133
14    140
15     25
16    148
17     80
18    117
19    138
Name: pre_clean_len, dtype: int64

In [5]:
# Trying the cleaning with an example tweet first.
df.text[3191]

'#MakeAmericaGreatAgain #Trump2016 https://t.co/6hP6PzPkY5'

In [6]:
example1 = BeautifulSoup(df.text[3191], 'lxml')
print(example1.get_text())

#MakeAmericaGreatAgain #Trump2016 https://t.co/6hP6PzPkY5


In [7]:
# Replacing links with "https"
print(df.text[1])
re.sub('https?://[A-Za-z0-9./]+','https',df.text[1])

RT @IvankaTrump: Thank you New Hampshire! ???? https://t.co/pdnxXq2jgy


'RT @IvankaTrump: Thank you New Hampshire! ???? https'

In [8]:
df.text[20:30]

20    Thank you Ohio! VOTE so we can replace Obamaca...
21    Join me live in Wilmington, Ohio!\nhttps://t.c...
22    If Obama worked as hard on straightening out o...
23    'ICE OFFICERS WARN HILLARY IMMIGRATION PLAN WI...
24    Thank you NH! We will end illegal immigration,...
25    Clinton Aides: ‘Definitely’ Not Releasing Some...
26    RT @TeamTrump: Mrs. Saucier's son is in prison...
27    "The Clinton Campaign at Obama Justice" #Drain...
28    Join me today in Wilmington, Ohio at 4pm: http...
29    There is no challenge too great, no dream outs...
Name: text, dtype: object

In [9]:
testing = df.text[226]
testing

'Join my team over on my Facebook page- live now! #Debates\nhttps://t.co/vpDVQfO58A https://t.co/5v1tWzHrxq'

In [10]:
testing.replace(u"\ufffd", "?")

'Join my team over on my Facebook page- live now! #Debates\nhttps://t.co/vpDVQfO58A https://t.co/5v1tWzHrxq'

In [11]:
# Removing special characters, leaving only letters.
re.sub("[^a-zA-Z]", " ", df.text[3191])

' MakeAmericaGreatAgain  Trump     https   t co  hP PzPkY '

In [12]:
# Actual method for cleaning the text
from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))


def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, 'https', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped

    # Do we want to do something for links etc?
    
    letters_only = re.sub("[^a-zA-Z]", " ", stripped)
    lower_case = letters_only.lower()

    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()

In [13]:
# Testing with small amount of texts
testing = df.text[:10]
test_result = []
for t in testing:
    test_result.append(tweet_cleaner(t))
test_result

['thank you iowa get out votetrumppence https https',
 'rt https thank you new hampshire https',
 'van jones there is a crack in the blue wall it has to do with trade https',
 'great night in denver colorado thank you together we will make america great again icymi watch rally here https',
 'rt https join https live in denver colorado via his facebook page we are here makeamericagreatagain https',
 'thank you reno nevada nothing will stop us in our quest to make america safe and great again americafirst https',
 'join me live in reno nevada https https',
 'join me tomorrow minnesota pm https michigan pm https virginia p https',
 'draintheswamp https',
 'top clinton aides bemoan campaign all tactics no vision https']

In [14]:
# Running the tweet_cleaner for the big data
print("Cleaning and parsing the tweets...\n")
clean_tweet_texts = []
for i in range(0, len(df)):
    if( (i+1)%1000 == 0 ):
        print(i+1, " tweets has been processed")                                                                  
    clean_tweet_texts.append(tweet_cleaner(df['text'][i]))

Cleaning and parsing the tweets...

1000  tweets has been processed
2000  tweets has been processed


/home/venla/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://t.co/ZQ0osiFEJQ" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


3000  tweets has been processed


/home/venla/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://t.co/SmTkLPiBYD" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


4000  tweets has been processed


...

10000  tweets has been processed


/home/venla/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://t.co/E3xvdUGZqa" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/venla/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://t.co/ue5JEZy85v" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/venla/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://t.co/yfwdyUHmn3" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.

11000  tweets has been processed


/home/venla/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://t.co/4OjDqTMEIx" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/venla/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://t.co/8lI2lomGkh" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/venla/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://t.co/6VLQYAlcto" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.

12000  tweets has been processed
13000  tweets has been processed


In [15]:
# Saving clean texts into a dataframe with timestamps
clean_df = pd.DataFrame(clean_tweet_texts,columns=['text'])
clean_df['timestamp'] = df.created_at
clean_df.head()

,text,timestamp
0,thank you iowa get out votetrumppence https https,"datetime.datetime(2016, 11, 6, 19, 26, 58)"
1,rt https thank you new hampshire https,"datetime.datetime(2016, 11, 6, 18, 9, 44)"
2,van jones there is a crack in the blue wall it...,"datetime.datetime(2016, 11, 6, 17, 6, 8)"
3,great night in denver colorado thank you toget...,"datetime.datetime(2016, 11, 6, 6, 8, 32)"
4,rt https join https live in denver colorado vi...,"datetime.datetime(2016, 11, 6, 4, 27)"


In [17]:
with open('../data/clean_tweets.json', 'w') as file:
    file.write(df.to_json())